## **Eduardo Carrasco Vidal** <img src="img/logo.png" align="right" style="width: 120px;"/>

**Magister en Inteligencia Artificial, Universidad Adolfo Ibáñez.**

**Profesor:** John Atkinson.
**Curso:** Procesamiento del Leguaje Natural (Natural Language Processing).

Enlace al repositorio del alumno en [GitHub](https://github.com/educarrascov/MIA_NaturalLP) _@educarrascov_

![Python](https://img.shields.io/badge/python-%2314354C.svg) 

**CLASIFICACIÓN DE SENTIMIENTOS CON LSA**

Este programa utiliza  *Análisis Semántico Latente*  (LSA) para generar [embeddings](https://es.wikipedia.org/wiki/Word_embedding) que permitan realizar posteriormente una actividad de clasificación de sentimientos.

La clasificación se realiza entrenando un modelo [Bayesiano Gausiano](https://iq.opengenus.org/gaussian-naive-bayes/).

Primero, necesitamos instalar algunos paquetes:

In [ ]:
#pip install es_core_news_sm

In [ ]:
import es_core_news_sm

In [ ]:
import os
import pandas as pd
import regex
import numpy as np
from scipy.spatial.distance import cosine
from spacy.lang.es.stop_words import STOP_WORDS
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import matplotlib.pyplot as plt
from numpy.linalg import svd

Definamos la función **CrearEspacioLSA(corpus,dim,NombreModelo)**, que  utiliza LSA para crear un *espacio semántico* de dimensiones reducidas (i.e., modelo vectorial que representa documentos y palabras), que se graba y luego puede ser cargado por otros programas. La función recibe la lista de documentos pre-procesados (**corpus**), el número de dimensiones a reducir (**dim**) vía [SVD](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491), y el nombre de la carpeta donde se grabará el modelo (**NombreModelo**).

LSA utiliza la *descomposición de valores singulares* (SVD) para descomponer la matriz de frecuencias original vía **TfidfVectorizer** en tres matrices: $U$, $\Sigma$ y $V^T$. Luego, las nuevas representaciones vectoriales en dimensiones reducidas (**dim**)  se reconstruyen  como:

*   Representación de  términos: $U(dim) *  \Sigma(dim)$

*   Representación  de documentos: $\Sigma(dim) * V^T(dim)$

In [ ]:
def CrearModeloLSA(textos,dim,NombreModelo):
  MatrizFrec = TfidfVectorizer()
  tf = MatrizFrec.fit_transform(textos).T
  U, Sigma, VT = svd(tf.toarray())
  # Se realiza producto punto de matrices con las nuevas dimensiones
  terms = np.dot(U[:,:dim], np.diag(Sigma[:dim]))
  docs  = np.dot(np.diag(Sigma[:dim]), VT[:dim, :]).T 
  vocab = MatrizFrec.get_feature_names()
  GrabarModeloLSA(NombreModelo, Sigma, terms, docs, vocab)

def GrabarModeloLSA(NombreModelo,Sigma,terms,docs, vocab):
   existe = os.path.isdir(NombreModelo)
   if not existe:
       os.mkdir(NombreModelo)
   joblib.dump(Sigma,   NombreModelo +"/"+'sigma.pkl') 
   joblib.dump(terms,   NombreModelo +"/"+'terms.pkl') 
   joblib.dump(docs,    NombreModelo +"/"+'docs.pkl') 
   joblib.dump(vocab,   NombreModelo +"/"+'vocab.pkl') 

def CargarModeloLSA(NombreModelo):
    terms   = joblib.load(NombreModelo+"/"+'terms.pkl')
    vocab   = joblib.load(NombreModelo+"/"+'vocab.pkl')
    modelo =  CrearDiccionario(terms,vocab)
    return(modelo)
def CrearDiccionario(lista_vectores,claves):
   dicc = {}
   for  v in range(0,len(claves)):
      dicc[claves[v]] = lista_vectores[v]
   return(dicc)

Un aspecto importante al utilizar LSA es determinar cuál es el número óptimo de dimensiones a reducir. Esto depende del tamaño del corpus utiizado para construir el modelo:

1.   Si el tamaño es muy grande (i.e., varios GigaBytes de texto), lo adecuado es utilizar entre 200 a 300 dimensiones.
2.   Si el tamaño  es pequeño, se puede elegir el número de dimensiones como el número de valores singulares de la matriz $\Sigma$) que maximice la *importancia* (descartando el primer valor pues corelaciona con el largo del corpus). La importancia de un valor singular $x$ es simplemente $x^2$.

Para visualizar la importancia de los valores singulares, podemos definir  la función **GraficarImportancia($\Sigma$)**, que grafica los valores singulares (i.e., dimensiones) versus importancia:

In [ ]:
def GraficarImportancia(Sigma):   
    NumValores = np.arange(len(Sigma))
    Importancia = [x**2 for x in Sigma]
    plt.bar(NumValores,Importancia)
    plt.ylabel('Importancia')
    plt.xlabel('Valores Singulares')
    plt.title('Importancia de Valores Singulares en SVD')
    plt.show()

Luego, definimos algunas funciones utilitarias para "limpieza" y pre-procesamieto:

In [ ]:
 def PreProcesar(textos):
    texto_limpio = []
    for texto in textos:  
        texto = Lematizar(texto)     
        texto = EliminaNumeroYPuntuacion(texto)      
        texto_limpio.append(texto)
    return(texto_limpio)

def Lematizar(oracion):
   doc = nlp(oracion)
   lemas = [token.lemma_ for token in doc]
   return(" ".join(lemas))  

def EliminaNumeroYPuntuacion(oracion):
    string_numeros = regex.sub(r'[\”\“\¿\°\d+]','', oracion)
    return ''.join(c for c in string_numeros if c not in punctuation)

def Tokenizar(oracion):
    doc = nlp(oracion)
    tokens = [palabra.text for palabra in doc]
    return(tokens)

def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  doc_id = []  
  for NombreArchivo  in directorio:
     try:
          texto = open(path+NombreArchivo,'r',encoding="utf-8").read()
          corpus.append(texto)
          doc_id.append(NombreArchivo)
     except IsADirectoryError:
          texto = ""
  return(corpus,doc_id)

Una vez que generamos las nuevas representaciones vectoriales para documentos y términos, podríamos utilizar dichos  vectores para realizar diversas tareas tales como clustering, clasificación, comparación de documentos, etc.

Como ejemplo, definamos la función **GraficarVectores(vocab,vectores)**, que toma vectores de documentos (o palabras), y el vocabulario, y grafica cada uno de los elementos en un gráfico 2-dimensional. Dado que los vectores contienen más de 2 dimensiones, en este ejemplo, sólo tomamos las 2 primeras. 

El gráfico le permitirá visualizar espacialmente la *cercanía* que existe entre documentos para posteriores análisis.

Alternativamente, Ud. podría aplicar otros métodos de reducción dimensional para graficas las mejores 2 componentes:

In [ ]:
def GraficarVectores(vocab,vectores):
    x = []
    y = []
    for value in vectores:
        x.append(value[0])
        y.append(value[1])   
    plt.figure(figsize=(7, 7))   
    plt.title("Distribución Espacial de Documentos")
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(vocab[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

Note que los vectores para documentos o palabras están indexados por posición y no por nombre, lo que podría dificultad el acceso. Para mejorar esto, podemos definir la función **CrearDiccionario(Vectores,vocabulario)**, que dado un conjunto de **vectores** (de documentos o palabras) y un **vocabulario**, genera un nuevo arreglo donde el índice es el nombre correspondiente: una palabra en el caso de vectores de palabras o un nombre de documento en el caso de vectores de documentos:

In [ ]:
def ObtenerEmbeddingOracion(modelo, dim, oracion):
   Lista_enbeddings = []
   Tokens = Tokenizar(oracion)
   for w in Tokens:
       # Verificar que la palabra w exista en el modelo
       try:
           modelo[w]
       except KeyError:
           continue
       # Obtener vector de la palabra w
       embedding = modelo[w]
       Lista_enbeddings.append(embedding)
   embedding_palabras = np.array(Lista_enbeddings)
   if (len(embedding_palabras) > 0):
        embedding_oracion = embedding_palabras.mean(axis=0)
   else:
        embedding_oracion = np.zeros(dim)
   return(embedding_oracion) 

Ahora, realizamos nuestro programa principal donde ajustamos la ruta donde se encuentran los documentos del corpus e inicializamos los modelos de lenguaje:

In [ ]:
PATH = "DiscursosOriginales/"
nlp = es_core_news_sm.load()

In [ ]:
# Preparando discursos de training y test
import shutil
from sklearn.model_selection import train_test_split
def SepararDiscursosTrainYtest(test_size=0.25):
    shutil.rmtree(PATH+'Train', ignore_errors=True)
    shutil.rmtree(PATH+'Test', ignore_errors=True)
    X = y = os.listdir(PATH)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
    os.mkdir(PATH+'Train')
    os.mkdir(PATH+'Test')
    for x in X_train:
        shutil.copyfile(PATH+x, PATH+'Train/'+x)
    for x in X_test:
        shutil.copyfile(PATH+x, PATH+'Test/'+x)
SepararDiscursosTrainYtest()

Creamos un corpus a partir de los documentos de la carpeta de *PATH*. Luego, creamos el espacio semántico utilizando LSA y lo grabamos para recuperarlo posteriormente. Por ahora podemos suponer que vamos a reducir el espacio original a $dim=3$ ($dim$ debe ser menor que el número de documentos): 

In [ ]:
dim =  200
if os.path.isdir("mi_lsa"):
    modeloLSA = CargarModeloLSA("mi_lsa")
else:
    corpus, lista_docs = CrearCorpus(PATH+'Train/')
    textos  = PreProcesar(corpus)
    CrearModeloLSA(textos,dim,"mi_lsa")
    modeloLSA = CargarModeloLSA("mi_lsa")

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
textos_test = os.listdir(PATH+'Test/')
textos_test

['95948.txt',
 '98069.txt',
 '90073.txt',
 '89375.txt',
 '91144.txt',
 '100227.txt',
 '102066.txt',
 '135819.txt',
 '100033.txt',
 '80384.txt',
 '84089.txt',
 '97896.txt',
 '100296.txt',
 '93927.txt',
 '152588.txt',
 '86848.txt',
 '75561.txt',
 '153298.txt',
 '164594.txt',
 '74634.txt',
 '75019.txt',
 '93848.txt',
 '101766.txt',
 '151434.txt',
 '100172.txt',
 '74598.txt',
 '81222.txt',
 '78838.txt',
 '100547.txt',
 '85341.txt',
 '73772.txt',
 '81517.txt',
 '101255.txt',
 '90435.txt',
 '92554.txt',
 '71901.txt',
 '80001.txt',
 '138176.txt',
 '149939.txt',
 '93108.txt',
 '81469.txt',
 '76217.txt',
 '151297.txt',
 '81311.txt',
 '88568.txt',
 '96720.txt',
 '86407.txt',
 '83512.txt',
 '85835.txt',
 '75320.txt',
 '74712.txt',
 '102687.txt',
 '164282.txt',
 '151972.txt',
 '72170.txt',
 '135207.txt',
 '148650.txt',
 '135757.txt',
 '93588.txt',
 '91265.txt',
 '94214.txt',
 '90274.txt',
 '84826.txt',
 '97684.txt',
 '150341.txt',
 '99397.txt',
 '101337.txt',
 '90064.txt',
 '91320.txt',
 '138138.t

In [ ]:
texto = open(PATH+'Test/100172.txt','r',encoding="utf-8").read()

In [ ]:
print(texto)

Muy buenas tardes:

 

Señor Ministro, señor Intendente, señor Senador, señor Presidente del Directorio, señor Alcalde; y quiero entregar un saludo muy especial a Nelson Pizarro, Presidente Ejecutivo de Codelco. Y, como Presidente de todos los chilenos, yo sé que interpreto a todos mis compatriotas al agradecerle, como lo hice en privado, muy sinceramente, más de medio siglo al servicio de nuestro país y más de tres décadas al servicio de Codelco.

 

Yo sé que ha puesto talento, pero eso no es mérito propio, viene desde el Creador. Quiero apreciar y agradecer la vocación, el compromiso y la entrega que dedicó a Codelco durante estas décadas y que nos permiten hoy día estar inaugurando una nueva etapa en la vida y en la historia de Codelco.

 

Quiero saludar también, y con mucho cariño y gratitud, a los trabajadores de Codelco que son los que con su inteligencia y su trabajo han hecho posible esta gran obra para Codelco y para Chile.

 

Y quiero hacer una reflexión. Ayer hubo un grav

In [ ]:
def resumir_texto(texto, lineas=0):
    nlp_text=nlp(texto)
    oraciones=[]
    for sent in nlp_text.doc.sents:
        oraciones.append(str(sent).replace('\xa0','').replace('\n',''))
    oraciones_new=[EliminaNumeroYPuntuacion(Lematizar(oracion)) for oracion in oraciones]
    features = [ObtenerEmbeddingOracion(modeloLSA, dim, oracion) for oracion in oraciones_new]
    features = np.array(features)
    print(f"El texto contiene {len(features)} parrafos")
    if lineas==0 or lineas<0:
        resumen=int(len(features)/2)
    else:
        resumen=lineas
    print(f"Se escogerán {resumen} parrafos para el resumen")
    similitudes = []
    vector_promedio= features.mean(axis=0)
    for feature in features: 
        similitud = 1-cosine(vector_promedio,feature)
        similitudes.append(similitud)
    print(f"Las similitudes encontradas son \n" + '\n'.join([str(x) for x in similitudes]))
    df = pd.DataFrame({'oraciones':oraciones, 'oraciones_new':oraciones_new, 'similitudes':similitudes})
    resumen_texto = [x for x in df.sort_values(by='similitudes').head(11).sort_index()['oraciones']]
    return resumen_texto

In [ ]:
resumen=resumir_texto(texto)

El texto contiene 67 parrafos
Se escogerán 33 parrafos para el resumen
Las similitudes encontradas son 
0.9880419673597625
0.9965828826723953
0.9922742256646414
0.9992838071453902
0.9984959554441492
0.9650700076724696
0.9956432115877422
0.9779236123022733
0.9979752744762229
0.9868167365300226
0.9990039924624349
0.9792802865648855
0.9968020465266652
0.9981745689073785
0.9980037713907138
0.9913495249225481
0.9994164369698021
0.9942955131380974
0.9986903918321726
0.9985647330866468
0.9983794378778291
0.9920432223806239
0.997056728246864
0.9957062826834518
0.9976021335623573
0.9994748987374275
0.9987270140559512
0.9837070435079452
0.998874255349748
0.9919748040069074
0.996373751940081
0.9994068927113317
0.9922608109973905
0.9981853349328861
0.99837952346809
0.9964845057893102
0.9939837420197674
0.9997438702546185
0.9530649248186409
0.9992760665412908
0.9985865661754786
0.9978581755186137
0.9990000920389475
0.9963914718777223
0.9991285209582269
0.9978124665998798
0.9975536224386018
0.998658

In [ ]:
print( '\n'.join(resumen))

Muy buenas tardes:Señor Ministro, señor Intendente, señor Senador, señor Presidente del Directorio, señor Alcalde; y quiero entregar un saludo muy especial a Nelson Pizarro, Presidente Ejecutivo de Codelco.
Y quiero hacer una reflexión.
Quiero decirles a esos seis chilenas y chilenos que perdieron sus vidas que nuestros pensamientos y nuestras oraciones están con sus familias, y, también, que está nuestro respaldo, nuestra solidaridad y nuestro apoyo.
Ese proyecto está en plena marcha y, de hecho, ya estaba trabajando en Valparaíso y en algunas regiones del Norte.
Hoy es un día importante para Codelco.
Y eso va a exigir mucha creatividad, mucha fuerza, mucha voluntad y mucha perseverancia porque siempre hay fuerzas que se oponen a los cambios.
Pero también sabemos que cuando nos dividimos en luchas fratricidas, muchas veces, también logramos nuestras más amargas derrotas.
Porque lo cierto es que estamos enfrentando exigencias distintas a las que conocíamos.
Pero también hay nuevos desa